In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import org.apache.log4j.{Level, Logger}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import java.sql.Timestamp
import java.time._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._

import $ivy.$
import org.apache.log4j.{Level, Logger}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import java.sql.Timestamp
import java.time._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._

In [2]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Hello Spark")
                .getOrCreate()
spark.sparkContext.setLogLevel("WARN")
import spark.implicits._
println(s"spark.version == ${spark.version}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/06/15 19:12:28 WARN Utils: Your hostname, MacBook-Air-Aroslav.local resolves to a loopback address: 127.0.0.1; using 192.168.1.222 instead (on interface en0)
24/06/15 19:12:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/06/15 19:12:29 INFO SparkContext: Running Spark version 3.5.0
24/06/15 19:12:29 INFO SparkContext: OS info Mac OS X, 14.4.1, aarch64
24/06/15 19:12:29 INFO SparkContext: Java version 11.0.23
24/06/15 19:12:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/15 19:12:29 INFO ResourceUtils: ==============================================================
24/06/15 19:12:29 INFO ResourceUtils: No custom resources configured for spark.driver.
24/06/15 19:12:29 INFO ResourceUtils: ==============================================================
24/06/15 19:12:29 INFO SparkContext: Submitt

spark.version == 3.5.0


spark: SparkSession = org.apache.spark.sql.SparkSession@6a4245e6
import spark.implicits._

In [3]:
val taxi_data_path= "resources/data/yellow_taxi_jan_25_2018"
val zone_data_path = "resources/data/taxi_zones.csv"

val schemaTaxi = StructType(
  Array(
    StructField("VendorID", IntegerType),
    StructField("tpep_pickup_datetime", TimestampType),
    StructField("tpep_dropoff_datetime", TimestampType),
    StructField("passenger_count", IntegerType),
    StructField("trip_distance", DoubleType),
    StructField("RatecodeID", IntegerType),
    StructField("store_and_fwd_flag", StringType),
    StructField("PULocationID", IntegerType),
    StructField("DOLocationID", IntegerType),
    StructField("payment_type", IntegerType),
    StructField("fare_amount", DoubleType),
    StructField("extra", DoubleType),
    StructField("mta_tax", DoubleType),
    StructField("tip_amount", DoubleType),
    StructField("tolls_amount", DoubleType),
    StructField("improvement_surcharge", DoubleType),
    StructField("total_amount", DoubleType)
  )
)
val baseDf = spark.read.schema(schemaTaxi).parquet(taxi_data_path)

val schemaZone = StructType(
    Array(
        StructField("LocationID", IntegerType),
        StructField("Borough", StringType),
        StructField("Zone", StringType),
        StructField("service_zone", StringType)
    )
)
val zoneDf = spark.read.schema(schemaZone).option("header", "true").csv(zone_data_path)
baseDf.cache()
baseDf.printSchema()
zoneDf.printSchema()
println("Data ready")

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)

Data ready


taxi_data_path: String = "resources/data/yellow_taxi_jan_25_2018"
zone_data_path: String = "resources/data/taxi_zones.csv"
schemaTaxi: StructType = Seq(
  StructField(
    name = "VendorID",
    dataType = IntegerType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "tpep_pickup_datetime",
    dataType = TimestampType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "tpep_dropoff_datetime",
    dataType = TimestampType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "passenger_count",
    dataType = IntegerType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "trip_distance",
    dataType = DoubleType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "RatecodeID",
    dataType = IntegerType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "store_and_fwd_flag",
...
baseDf: org.apache.spark.sql.package.DataFrame = [VendorID: int, tpep_pickup_datetime: timest

In [7]:
val task1 = baseDf.join(zoneDf, baseDf("DOLocationID") === zoneDf("LocationID"))
  .groupBy("Zone").count().withColumnRenamed("count", "trips_number")
  .orderBy(col("trips_number").desc)
task1.show(10, false)
/* 
+-------------------------+------------+
|Zone                     |trips_number|
+-------------------------+------------+
|Midtown Center           |15099       |
|Upper East Side North    |14261       |
|Upper East Side South    |13754       |
|Murray Hill              |11239       |
|Midtown East             |11090       |
|Times Sq/Theatre District|10054       |
|Union Sq                 |9929        |
|Lincoln Square East      |8666        |
|Midtown North            |8594        |
|Clinton East             |8258        |
+-------------------------+------------+
only showing top 10 rows
*/
task1.write.mode("overwrite").parquet("result/task1/task1.parquet")

+-------------------------+------------+
|Zone                     |trips_number|
+-------------------------+------------+
|Midtown Center           |15099       |
|Upper East Side North    |14261       |
|Upper East Side South    |13754       |
|Murray Hill              |11239       |
|Midtown East             |11090       |
|Times Sq/Theatre District|10054       |
|Union Sq                 |9929        |
|Lincoln Square East      |8666        |
|Midtown North            |8594        |
|Clinton East             |8258        |
+-------------------------+------------+
only showing top 10 rows



task1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Zone: string, trips_number: bigint]

In [12]:
val task2 = baseDf.groupBy("tpep_pickup_datetime").count().withColumnRenamed("count", "trips_number")
                  .orderBy(col("trips_number").desc)
                  .select(col("tpep_pickup_datetime"),
                           col("trips_number"))
task2.show(10, false)
val taskText = task2.rdd.map(row => s"${row(0).toString},${row(1).toString}\n").collect()

import java.io.File
val dir = new File("result/task2")
if (!dir.exists()) {
  dir.mkdirs()
}

val file = new java.io.FileWriter("result/task2/task2.txt")
file.write(taskText.mkString)
file.close()

+--------------------+------------+
|tpep_pickup_datetime|trips_number|
+--------------------+------------+
|2018-01-25 09:51:56 |18          |
|2018-01-25 19:48:51 |17          |
|2018-01-25 19:54:47 |17          |
|2018-01-25 14:32:01 |17          |
|2018-01-25 22:38:40 |17          |
|2018-01-25 18:24:54 |16          |
|2018-01-25 19:11:41 |16          |
|2018-01-25 09:25:26 |16          |
|2018-01-25 20:25:25 |15          |
|2018-01-25 19:01:49 |15          |
+--------------------+------------+
only showing top 10 rows



task2: org.apache.spark.sql.package.DataFrame = [tpep_pickup_datetime: timestamp, trips_number: bigint]
taskText: Array[String] = Array(
  """2018-01-25 09:51:56.0,18
""",
  """2018-01-25 19:48:51.0,17
""",
  """2018-01-25 19:54:47.0,17
""",
  """2018-01-25 14:32:01.0,17
""",
  """2018-01-25 22:38:40.0,17
""",
  """2018-01-25 18:24:54.0,16
""",
  """2018-01-25 19:11:41.0,16
""",
  """2018-01-25 09:25:26.0,16
""",
  """2018-01-25 09:46:21.0,15
""",
  """2018-01-25 20:57:23.0,15
""",
  """2018-01-25 15:39:44.0,15
""",
  """2018-01-25 19:01:49.0,15
""",
  """2018-01-25 22:26:49.0,15
""",
  """2018-01-25 18:10:51.0,15
""",
  """2018-01-25 21:17:03.0,15
""",
  """2018-01-25 20:25:25.0,15
""",
  """2018-01-25 20:00:19.0,15
""",
  """2018-01-25 20:00:14.0,15
""",
  """2018-01-25 18:57:08.0,15
""",
...
import java.io.File
dir: File = result/task2
res12_5: AnyVal = true
file: java.io.FileWriter = java.io.FileWriter@3a7ebefb

In [13]:
val task3 = baseDf.groupBy("trip_distance").count()
                  .withColumnRenamed("count", "trips_number")
                  .orderBy(col("trips_number").desc)
                  .as[(String, Long)]

task3.show(10, false)
task3.write.mode("overwrite").parquet("result/task3/task3.parquet")

+-------------+------------+
|trip_distance|trips_number|
+-------------+------------+
|0.8          |8223        |
|0.7          |8156        |
|0.9          |8102        |
|1.0          |7821        |
|1.1          |7453        |
|0.6          |7422        |
|1.2          |6895        |
|1.3          |6514        |
|1.4          |5979        |
|0.5          |5958        |
+-------------+------------+
only showing top 10 rows



task3: org.apache.spark.sql.Dataset[(String, Long)] = [trip_distance: double, trips_number: bigint]

In [ ]:
base_df.unpersist()
    spark.stop()